In [1]:
import pandas as pd
import datetime
from dateutil import tz

import ee
import geemap

pd.options.mode.copy_on_write = True

Инициализация GEE

In [2]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')
baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

(node:22530) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


Приведение к читаемому виду

In [3]:
def add_timezone(df: pd.DataFrame) -> pd.DataFrame:
   # format = "%d.%m.%Y  %H:%M:%S"
   format = "%Y-%m-%d  %H:%M:%S"
   return df["Time"].apply(lambda dt: pd.to_datetime(dt, format=format).replace(tzinfo=tz.gettz("Asia/Irkutsk")))

In [4]:
df_lst = pd.read_csv('Наземеные_измерения/ready/Temp21-24_cut15.csv')
df_lst["Time"] = add_timezone(df_lst)
df_lst["T"] = df_lst['T'].astype(float)
df_lst["Lat"] = df_lst['Lat'].astype(float)
df_lst["Lon"] = df_lst['Lon'].astype(float)
df_lst

,Time,T,Lat,Lon
0,2021-05-25 18:42:01+08:00,2.56830,51.896156,105.098007
1,2021-05-25 18:57:03+08:00,2.26880,51.887924,105.153023
2,2021-05-25 19:12:08+08:00,2.32920,51.897236,105.206261
3,2021-05-25 19:27:10+08:00,2.34780,51.909248,105.258759
4,2021-05-25 19:42:15+08:00,2.36390,51.920834,105.312088
...,...,...,...,...
17661,2024-10-03 13:39:00+08:00,9.72080,51.847950,104.870740
17662,2024-10-03 13:54:00+08:00,9.80560,51.847950,104.870730
17663,2024-10-03 14:09:00+08:00,9.86995,51.847950,104.870730
17664,2024-10-03 14:24:00+08:00,9.81650,51.847940,104.870730


In [ ]:
date_start = df_lst['Time'].min()
date_end = df_lst['Time'].max()

In [ ]:
def M11_scale_to_celsius(modis_image: ee.image):
   scaled_lsts = modis_image.select(['LST_Day_1km', 'LST_Night_1km']).multiply(0.02).subtract(273.15).rename(['LST_Day', 'LST_Night'])
   return modis_image.addBands(srcImg=scaled_lsts)

def M21_scale_to_celsius(modis_image: ee.Image):
   scaled_lsts = modis_image.select(['LST_Day', 'LST_Night']).subtract(273.15)
   return modis_image.addBands(srcImg=scaled_lsts, overwrite=True)

def M_get_image_list(satellite_name: str):
   match satellite_name:
      case 'MOD11_Terra':
         coll = ee.ImageCollection('MODIS/061/MOD11A1').map(lambda img: M11_scale_to_celsius(img))
      case 'Modis21_Terra':
         coll = ee.ImageCollection('MODIS/061/MOD21C1').map(lambda img: M21_scale_to_celsius(img))
      case 'MOD11_Aqua':
         coll = ee.ImageCollection('MODIS/061/MYD11A1').map(lambda img: M11_scale_to_celsius(img))
      case 'Modis21_Aqua':
         coll = ee.ImageCollection('MODIS/061/MYD21C1').map(lambda img: M21_scale_to_celsius(img))

   coll = coll.filter(ee.Filter.date(date_start, date_end)).filterBounds(baikal_shape)
   if(coll.size().getInfo() == 0):
      return None
   coll = coll.toList(coll.size())
   return coll

In [ ]:
def get_time(image: ee.Image, is_day: bool, is_M21: bool):
   date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
   image_dt = pd.to_datetime(date, format='%Y-%m-%d')

   info = image.reduceRegion(
         reducer=ee.Reducer.firstNonNull(),
         geometry=baikal_shape
         ).getInfo()
   if (is_day):
      time = info["Day_view_time"] #This band stores hours with 0.1 scale
   else: 
      time = info["Night_view_time"] #This band stores hours with 0.1 scale
   
   if(time is None):
      return None

   if (is_M21):
      timezone = tz.UTC
   else:
      timezone = tz.gettz("Asia/Irkutsk")
      time = time / 10

   time = datetime.timedelta(hours=time)
   image_dt = image_dt.replace(tzinfo=timezone, hour=time.seconds//3600, minute=(time.seconds//60)%60)
   
   return image_dt

def find_entries_for_image(
      image: ee.Image, 
      df: pd.DataFrame, 
      minutes_padding: int, 
      is_day: bool, 
      is_M21: bool) -> pd.DataFrame:
   image_dt = get_time(image, is_day, is_M21)
   if(image_dt is None):
      # Нет дневных или ночных данных
      return None

   margin = pd.Timedelta(minutes_padding, 'minutes')
   start_dt = image_dt - margin
   end_dt = image_dt + margin

   print(f'Начало: {start_dt},\nКонец: {end_dt}')
   
   close_entries = df[(start_dt <= df['Time']) & (df['Time'] <= end_dt)]
   close_entries['Image Time'] = image_dt
   return close_entries

def get_validation_entry(
      image: ee.Image, 
      df: pd.DataFrame, 
      minutes_padding: int,
      is_day: bool, 
      is_M21: bool) -> pd.Series:
   found_entries = find_entries_for_image(image, df, minutes_padding, is_day, is_M21)
   if(found_entries is None or found_entries.shape[0] == 0): # Если нет строк - не нашли подходящих измерений
      print("Не найдены измерения")
      return None
   print(f"Найдено {found_entries.shape[0]} измерений")
   
   good_entry = None
   for _, entry in found_entries.iterrows():
      longtitude = entry['Lon']
      latitude = entry['Lat']

      region = ee.Geometry.Point([longtitude, latitude])
      image_averaged_dict = image.reduceRegion(
         reducer=ee.Reducer.mean(),
         geometry=region
      ).getInfo()

      if (is_day):
            image_temperature = image_averaged_dict['LST_Day']
      else:
            image_temperature = image_averaged_dict['LST_Night']

      if(image_temperature is not None):
         # Нашли подходящее измерение
         good_entry = entry
         print(f'Долгота: {longtitude},\nШирота: {latitude}')
         break

   if(good_entry is None):
      return None
   
   good_entry['Image T'] = image_temperature
   return good_entry

def compute_temperatures(
      images: ee.List, 
      df: pd.DataFrame, 
      minutes_padding: int, 
      is_day: bool, 
      is_M21: bool) -> pd.DataFrame:
   result_list = []
   
   size = images.size().getInfo()
   for i in range(0, size):
      print(f"Снимок #{i} из {size}")
      image = ee.Image(images.get(i))
      entry = get_validation_entry(image, df, minutes_padding, is_day, is_M21)
      if(entry is None):
         # это происходит, например когда во время снятия снимка измерений не было
         continue
      elif(entry['Image T'] is None):
         # координата, где было сделано измерение, не попала в снимок (хоть по датам всё сходится)
         print("Координаты измерений лежат вне снимка")
         continue

      print(f"Замеренная температура: {entry['T']}")
      print(f"Предсказанная температура: {entry['Image T']}")
      result_list.append(entry)
      
   return pd.DataFrame(result_list)

def get_validation_data( 
      satellite_name: str,
      is_day: bool, 
      minutes_padding: int = 15):
   coll = M_get_image_list(satellite_name)
   if(coll is None):
      # image collection is empty
      return None
   
   if(satellite_name == 'MOD21_Terra' or satellite_name == 'MOD21_Aqua'):
      is_M21 = True
   else:
      is_M21 = False

   computed_df = compute_temperatures(
      coll, 
      df_lst, 
      minutes_padding, 
      is_day, 
      is_M21)
   
   computed_df['MODIS'] = satellite_name
   if(is_day):
      computed_df['Day/Night'] = 'Day'
   else:
      computed_df['Day/Night'] = 'Night'

   return computed_df

def get_validation_dataframe(minutes_padding: int) -> pd.DataFrame:
   dataframe_list = []

   modises = ['MOD11_Terra', 'MOD11_Aqua', 'MOD21_Terra', 'MOD21_Aqua']
   for modis in modises:
      print("----------")
      print(modis)
      for is_day in [True, False]:
         print("\tДневное время") if is_day else print("\tНочное время")
         modis_df = get_validation_data(modis, is_day, minutes_padding=minutes_padding)
         if(modis_df is None):
            print("Пустая коллекция")
            break
         dataframe_list.append(modis_df)

   return pd.concat(dataframe_list, ignore_index=True)

In [8]:
result_15 = get_validation_dataframe(15)
result_15

----------
Modis11_Terra
	Дневное время
Снимок #0
Начало: 2021-05-26 11:45:00+08:00,
Конец: 2021-05-26 12:15:00+08:00
Найдено 2 измерений
Снимок #1
Начало: 2021-05-27 10:51:00+08:00,
Конец: 2021-05-27 11:21:00+08:00
Найдено 2 измерений
Долгота: 106.586891,
Широта: 52.465443
Замеренная температура: 2.228
Предсказанная температура: 2.3500000000000227
Снимок #2
Начало: 2021-05-28 11:33:00+08:00,
Конец: 2021-05-28 12:03:00+08:00
Найдено 2 измерений
Долгота: 105.97818,
Широта: 52.052254
Замеренная температура: 1.42
Предсказанная температура: 1.6900000000000546
Снимок #3
Начало: 2021-05-29 10:33:00+08:00,
Конец: 2021-05-29 11:03:00+08:00
Найдено 2 измерений
Снимок #4
Начало: 2021-05-30 11:21:00+08:00,
Конец: 2021-05-30 11:51:00+08:00
Не найдены измерения
Снимок #5
Начало: 2021-05-31 10:27:00+08:00,
Конец: 2021-05-31 10:57:00+08:00
Не найдены измерения
Снимок #6
Начало: 2021-06-01 11:03:00+08:00,
Конец: 2021-06-01 11:33:00+08:00
Не найдены измерения
Снимок #7
Начало: 2021-06-02 10:15:00+08:00

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,Modis11_Terra,Day
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,Modis11_Terra,Day
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,Modis11_Terra,Day
3,2021-06-04 11:49:31+08:00,2.57930,51.626816,105.359306,2021-06-04 11:54:00+08:00,4.11,Modis11_Terra,Day
4,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,Modis11_Terra,Day
...,...,...,...,...,...,...,...,...
807,2024-09-21 03:25:00+08:00,10.60505,51.876760,105.205520,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night
808,2024-09-22 03:58:00+08:00,11.99560,52.100490,106.120810,2024-09-21 20:12:00+00:00,11.61,Modis21_Aqua,Night
809,2024-09-23 05:01:00+08:00,11.86775,52.575820,106.606930,2024-09-22 20:48:00+00:00,10.59,Modis21_Aqua,Night
810,2024-09-25 03:36:00+08:00,12.23610,53.521020,108.145930,2024-09-24 19:48:00+00:00,9.59,Modis21_Aqua,Night


In [ ]:
result_30 = get_validation_dataframe(30)
result_30

----------
Modis11_Terra
	Дневное время
Снимок #0
Начало: 2024-05-30 10:12:00+08:00,
Конец: 2024-05-30 11:12:00+08:00
Найдено 61 измерений
Долгота: 103.8697950819672,
Широта: 51.67456672131147
Замеренная температура: 2.967525868852459
Предсказанная температура: 2.7100000000000364
Снимок #1
Начало: 2024-05-31 10:54:00+08:00,
Конец: 2024-05-31 11:54:00+08:00
Найдено 61 измерений
Долгота: 104.42872459016394,
Широта: 51.64633311475411
Замеренная температура: 2.7601766065573767
Предсказанная температура: 3.4700000000000273
Снимок #2
Начало: 2024-06-01 09:54:00+08:00,
Конец: 2024-06-01 10:54:00+08:00
Найдено 61 измерений
Долгота: 105.30891475409837,
Широта: 51.88718770491804
Координаты измерений лежат вне снимка
Снимок #3
Начало: 2024-06-02 09:00:00+08:00,
Конец: 2024-06-02 10:00:00+08:00
Не найдены измерения
Снимок #4
Начало: 2024-06-03 11:12:00+08:00,
Конец: 2024-06-03 12:12:00+08:00
Найдено 4 измерений
Долгота: 105.79769999999999,
Широта: 52.232369999999996
Координаты измерений лежат вне 

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night
0,2024-05-30 10:42:00+08:00,2.967526,51.674567,103.869795,2024-05-30 10:42:00.000000256+08:00,2.71,Modis11_Terra,Day
1,2024-05-31 11:23:59.999999744+08:00,2.760177,51.646333,104.428725,2024-05-31 11:23:59.999999232+08:00,3.47,Modis11_Terra,Day
2,2024-06-04 10:48:00+08:00,2.719412,52.567929,106.339500,2024-06-04 10:48:00+08:00,3.31,Modis11_Terra,Day
3,2024-06-07 11:11:59.999999744+08:00,2.806782,53.890291,108.639307,2024-06-07 11:12:00+08:00,2.83,Modis11_Terra,Day
4,2024-06-08 10:11:59.999999744+08:00,2.205389,54.451103,109.072043,2024-06-08 10:12:00.000000768+08:00,2.73,Modis11_Terra,Day
...,...,...,...,...,...,...,...,...
182,2024-09-21 03:26:26.153846528+08:00,10.990785,51.873811,105.202675,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night
183,2024-09-22 04:11:59.999999744+08:00,11.982016,52.100502,106.120768,2024-09-21 20:11:59.999999232+00:00,11.61,Modis21_Aqua,Night
184,2024-09-23 04:47:59.999999744+08:00,11.766602,52.550601,106.582422,2024-09-22 20:48:00+00:00,10.57,Modis21_Aqua,Night
185,2024-09-25 03:47:59.999999744+08:00,11.852942,53.546656,108.141289,2024-09-24 19:48:00.000000768+00:00,9.59,Modis21_Aqua,Night


In [ ]:
result_15.to_csv('Данные_валидации/raw/modis_validation15.csv', index=False)
result_15.to_csv('Данные_валидации/ready/modis_validation15.csv', index=False)

M11 занижает t, M21 завышает t